In [9]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from tensorflow.keras import layers
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
import statistics
from numpy import array
from numpy import asarray
from numpy import zeros
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras import layers
import tensorflow as tf
from tensorflow import keras
import contractions
import re
import transformers
from transformers import BertTokenizer, TFBertForSequenceClassification
train = pd.read_csv(r'C:\Users\diego\OneDrive\Desktop\DS\Untitled Folder 1\emotions_dataset\train.txt', names=['sentences', 'emotion'], sep=';')
def Lemmatizer_stop_word(sentence):
    stop_words = set(stopwords.words('english'))
    lemmatizer = WordNetLemmatizer() #look at other Lemmatizers and stemmers
    sentence = re.sub('[^A-z]', ' ', sentence)
    negative = ['not', 'neither', 'nor', 'but', 'however',
                'although', 'nonetheless', 'despite', 'except',
                        'even though', 'yet']
    stop_words = [z for z in stop_words if z not in negative]
    preprocessed_tokens = [lemmatizer.lemmatize(contractions.fix(temp.lower())) for temp in sentence.split() if temp not in stop_words] #lemmatization
    return ' '.join([x for x in preprocessed_tokens]).strip()
max_length=43
from transformers import AutoTokenizer, TFBertModel
tokenizer=AutoTokenizer.from_pretrained('bert-base-cased')
bert=TFBertModel.from_pretrained('bert-base-cased')
lb = LabelEncoder()
labels_train=lb.fit(train.loc[:,'emotion'].to_list())
tf.random.set_seed(79)
max_length = 43
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.utils import to_categorical
input_ids = Input(shape=(max_length,), dtype=tf.int32, name="input_ids")
input_mask = Input(shape=(max_length,), dtype=tf.int32, name="attention_mask")
# embeddings = dbert_model(input_ids,attention_mask = input_mask)[0]
embeddings = bert(input_ids,attention_mask = input_mask)[0] #(0 is the last hidden states,1 is the pooler_output)
x = tf.keras.layers.GlobalMaxPool1D()(embeddings)
x = Dense(138, activation='elu',kernel_initializer='GlorotNormal')(x)
x = tf.keras.layers.Dropout(0.1)(x)
x = Dense(28,activation = 'elu',kernel_initializer='GlorotNormal')(x)
output = Dense(6,activation = 'softmax')(x)
model_saved = tf.keras.Model(inputs=[input_ids, input_mask], outputs=output)
model_saved.layers[2].trainable = True
model_saved.load_weights('Bert_stepword_lemmatizer.h5')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\diego\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\diego\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If yo

# <font color='violet'>Predict emotion from a new sentence</font>

In [26]:
y=input()



I am astonished by what you've accomplished


In [27]:
y_s=pd.Series([y])
y_lemm=y_s.apply(lambda x: Lemmatizer_stop_word(x))
y_tok = tokenizer(
    [x.split() for x in y_lemm],
    add_special_tokens=True,
    max_length=max_length,
    truncation=True,
    padding='max_length',  #only for class prediction from sentence
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    is_split_into_words=True,
    verbose = True)
#labels_y=lb.transform(test.loc[:,'emotion'].to_list())
y_prob=model_saved.predict({'input_ids':y_tok['input_ids'],'attention_mask':y_tok['attention_mask']})*100
#y_tok
class_label=y_prob.argmax(axis=-1)
lb.inverse_transform(class_label) #from class to label

array(['surprise'], dtype='<U8')